Contenido bajo licencia Creative Commons BY 4.0 y código bajo licencia MIT. © Juan Gómez y Nicolás Guarín-Zapata 2020. Este material es parte del curso Modelación Computacional en el programa de Ingeniería Civil de la Universidad EAFIT.

# Introducción al Análisis Estructural

## Introducción

En este Notebook se introduce de manera intuitiva el concepto de rigidez mediante el problema simple de un resorte sometido a una carga constante. Posteriormente, la idea se extiende al caso de un sólido (o medio continuo). Luego, se sume que el el sólido está conformado por pedazos triangulares, cada uno de ellos definido por 3 "partículas", conectadas por "resortes". Esta idealización permite representar el sólido como un sistema de partículas conectadas por resortes. Una vez el problema se reduce a un sistema de masas-resortes se hace evidente que la solución de ambos sistemas usa el mismo tipo de algoritmo. En la parte final del Notebook se implementa un programa general para resolver sistemas representados en términos de relaciones fuerza-desplazamiento.


**Al completar este notebook usted debería estar en la capacidad de:**

* Entender el concepto generalizado de rigidez tanto en sistemas continuos como discretos.

* Usar razonamientos mecánicos para resolver problemas de equilibrio (estático) conformados por partículas conectadas por resortes.

* Reconocer la equivalencia entre el problema de un sólido sometido a cargas externas y el de un sistema de masas-resortes.

* Entender los pasos  para resolver problemas de sistemas masa-resorte o su equivalente.




### Metodología para usar el notebook



El presente notebook esta diseñado para ser usado bajo una metodología de aprendizaje activo y mediante un esquema de clase invertida. El NB está conformado por una parte teórica y una parte computacional. Para poder alcanzar de manera satisfactoria los objetivos de aprendizaje es necesario que antes de la clase presencial el estudiante lea con detalle los contenidos teóricos del NB.

<div class="alert alert-warning">
Los bloques resaltados en amarillo corresponden a actividades para la clase, las cuales en algunos casos deben ser realizadas con el apoyo del profesor.
</div>

## Concepto intuitivo de rigidez


Considere los 2 modelos mostrados en la siguiente figura. El primero corresponde a un resorte de constante $k$ el cual tras ser sometido a la acción de una fuerza externa $F$ experimenta un cambio en su longitud igual a $\delta$. El mismo problema se ilustra en el modelo abstracto de un sólido representado por la "papa" y en el cual la línea continua describe la forma de la "papa" antes de aplicar la fuerza externa $F$, mientras que la línea punteada describe la forma de la "papa" tras aplicar la fuerza. Los apoyos esquemáticos (de triángulo y rodillo) que se muestran en la figura indican que la "papa" no se podrá desplazar como un cuerpo rígido.

<center>
    <img src="img/rigidez.svg"
         alt="Ejemplo de rigidez"
         style="width:800px">
</center>


En el resorte se satisface la relación

\begin{equation}
F = k \delta
\end{equation}

y en la cual el coeficiente de rigidez $k$ representa la fuerza necesaria para producir un desplazamiento unitario $\delta = 1.0$.

En el caso de la "papa", el problema es mas complejo ya que existen muchas direcciones posibles y puntos en los cuales aplicar las fuerzas y en los cuales medir los desplazamientos. ¿Cómo podemos cuantificar la rigidez para este caso, entonces?

Sea $\delta _i$ el desplazamiento en un punto cualquiera medido en una dirección $i$ y sea $F_j$ la fuerza aplicada en la dirección $j$ también en un punto arbitrario de la "papa". En este caso siguiendo la idea del resorte es posible escribir:

\begin{equation}
F_j = k \delta_i\, .
\end{equation}

Comparando esta relación con la correspondiente al problema del resorte se puede concluir que el coeficiente $k$ representa la fuerza necesaria a lo largo de la dirección $j$ para producir un desplazamiento unitario en la dirección $i.$


Nótese que si se cambia la dirección en la que se aplica la fuerza y la dirección en la que se mide el desplazamiento es esperable que el coeficiente de rigidez también cambie. Por lo tanto un modelo mas general sería:

\begin{equation}
F_j = k_{ji} \delta_i\, .
\end{equation}

y en el cual los subíndices $ij$ dan cuenta de que el coeficiente de rigidez $k_{ij}$ representa la fuerza necesaria a lo largo de la dirección $j$ para producir un desplazamiento unitario en la dirección $i.$

### Como describir el sólido?

El modelo propuesto para el caso del sólido es físicamente correcto pero operativamente existen infinitas direcciones posibles $i$ y $j$ y surge entonces la pregunta: **¿cómo cuantificar la rigidez para el caso de la "papa"?**

Consideremos algunas posibles soluciones:

1.  Solución discreta: Aproximar el sólido mediante un sistema finito de partículas conectadas por resortes.

2. Solución del continuo: Estudiar el comportamiento de un elemento diferencial generando un sistema de ecuaciones diferenciales (en las variables $u_i$, $\epsilon_{ij}$ y $\sigma_{ij}$).

3. Solución numérica: Combinar **1** y **2** para hacer una aproximación discreta del continuo (métodos por elementos finitos). La comparación entre las soluciones **1** y **2** se describe en la siguiente tabla:

<div class="alert alert-warning">

Discutir con sus compañeros de grupo las posibles ventajas y desventajas de las soluciones **1** a la **3**.

</div>







| Planteamiento del continuo | Planteamiento discreto |
|:--------------------------:|:----------------------:|
| Número infinito de elementos diferenciales. | Número finito de partículas conectadas por resortes. |
| Desplazamientos, deformaciones y tensiones. | Desplazamientos y fuerzas. |

### Elementos finitos (aproximando el continuo).





La figura muestra nuevamente el problema esquemático de la "papa". Esta ha sido dividida en un numero **finito** de triángulos conectados en las esquinas. Cada triángulo esta definido por 3 puntos correspondientes a sus vértices. Denominemos un triangulo típico como $\Omega_e$. Asuma que cada vértice del triángulo típico $\Omega_e$ se comporta como una partícula y que esta puede experimentar desplazamientos (y fuerzas) en la dirección horizontal y vertical. Asuma además que para el triángulo típico las 3 partículas que lo definen están conectadas por resortes.

<div class="alert alert-warning">

¿Cuántos coeficientes del tipo $k_{ij}$ existen para cada triángulo?

</div>


<center>
    <img src="img/dominio_discreto.svg"
         alt="Dominio discretizado."
         style="width:400px">
</center>

### Relación fuerza-desplazamiento para todo el sistema



Se tiene entonces que para un triangulo típico $\Omega_e$ es posible relacionar las 6 fuerzas con los 6 desplazamientos mediante una ecuación general como:

\begin{equation}
F_j = k_ {ji} \delta_i
\end{equation}

y en la cual el término $k_{ji}$ representa 36 coeficientes de rigidez. Si se considera que el índice $j$ puede tomar 6 valores correspondientes a 6 fuerzas y que el índice $i$ puede tomar 6 valores correspondientes a 6 desplazamientos entonces se puede reconocer que se tiene un sistema de 6 ecuaciones el cual podemos escribir matricialmente como:

\begin{equation}
\{ F \}=[ K ] \{ u \}\, .
\end{equation}

De manera similar, es posible anticipar que es posible escribir una relación que considere todos los puntos del dominio. Por ejemplo, si se tienen $N$ puntos en total habrán $2N$ fuerzas y $2N$ desplazamientos. Escribamos la relación entre fuerzas y desplazamientos para todo el sólido (representado por $N$ puntos) como:

\begin{equation}
F_j^G = k_ {ji}^G \delta_i\, .
\end{equation}

En esta relación hemos usado el superíndice $G$ para enfatizar que las fuerzas, desplazamientos y coeficientes de rigidez corresponden a toda el sólido y no a un elemento típico. En forma matricial se tiene:

\begin{equation}
\{ F^G \} = [ K^G ] \{ U^G \}\, .
\end{equation}

<div class="alert alert-warning">

¿Cuántos coeficientes del tipo $k_{ij}$ existen para todo el sólido si este esta representado por $N$ puntos?

</div>

El superínidce $G$ también indica que las ecuaciones son válidas para todo el sistema global representado por todos los triángulos en los que ha sido dividido el sólido.

<div class="alert alert-warning">
    
* Identifique el argumento de tipo físico que permite formar el sistema global de ecuaciones:

\begin{equation}
F_j ^ G = k_{ji} ^ G \delta_i\, .
\end{equation}

* Si el sólido se divide en $M$ triángulos unidos a través de un total de $N$ vértices, ¿cuántos desplazamientos del tipo $\delta_i$ hay en el sistema?

* Si el sólido se divide en $M$ triángulos unidos a través de un total de $N$ vértices, ¿cuántas fuerzas del tipo $F_j ^ G$ hay en el sistema?
</div>

De acuerdo con la discusión anterior, tanto el equilibrio de un triángulo típico $\Omega_e$ como el de un resorte pueden formularse mediante una relación fuerza-desplazamiento de la forma:

\begin{equation}
\{ F \} = [ K ] \{ u \}\, .
\end{equation}

En el caso del resorte es sencillo determinar la matriz de rigidez $[K]$.

En el caso del sólido el problema es un poco más elaborado y por el momento nos conformaremos con suponer que esta matriz se encuentra disponible. En cualquier caso, sea que se trata de un sistema de partículas conectadas por resortes, o un sólido representado por triángulos, el problema se resuelve ensamblando la matriz de rigidez global $[K^G]$ y resolviendo el sistema de ecuaciones:

\begin{equation}
\{ F^G \} = [ K^G ] \{ U^G \}\, .
\end{equation}


En la siguiente sección se discute con más detalle el problema usando un sistema de partículas unidas por resortes y se implementa su solución en el computador.

<div class="alert alert-warning">
    
### Sistema particula-resorte

    
La siguiente actividad permite entender desde un punto de vista físico el proceso de solución del problema de varias partículas interactuando a través de un sistema de resortes como el que se muestra en la figura.

<center>
<img src="img/sistema_resortes.svg"
     alt="Sistema de masas y resortes"
     style="width:400px">
</center>

El sistema de varias partículas se encuentra conectado por resortes con coeficiente de rigidez $k$. El sistema se encuentra sometido a diferentes fuerzas externas (representadas en múltiplos de $P$). Por efectos de visualización las partículas se representan como carritos rectangulares.

Se requiere:

1. Plantear las ecuaciones de equilibrio para una partícula típica mostrada en la figura. Asuma que la partícula esta conectada a un resorte $i$ por la izquierda y a un resorte $i+1$ por la derecha de manera que el diagrama de cuerpo libre para la partícula es como el que se muestra en la figura.

<center>
<img src="img/dcl_masa.svg"
     alt="Diagrama de cuerpo libre para una partícula"
     style="width:200px">
</center>



2. Plantear las relaciones fuerza-desplazamiento para el resorte típico mostrado en la figura. En esta los subíndices 1 y 2 hacen referencia a las fuerzas y desplazamientos de los puntos 1 y 2 que definen el resorte.

<center>
    <img src="img/elemento_resorte.svg"
         alt="Balance de fuerzas para un resorte"
         style="width:300px">
</center>

3. Enumerar las partículas y resortes del sistema. (Asumir el empotramiento como una partícula).

4. Escribir las ecuaciones de equilibrio para cada una de las 4 partículas.

5. Usando las relaciones fuerza-desplazamiento de los diferentes resortes re-escribir las ecuaciones del numeral **4**.

</div>

## Algoritmo de solución

En la solución del problema anterior se usaron planteamientos mecánicos para formular las ecuaciones de equlibrio del sistema de partículas. Consideremos ahora su solución de una manera sistemática de tal forma que el problema se pueda codificar en un programa general de análisis estructural.


### Equilbrio de los resortes

Considere el siguiente sistema de 3 partículas, las cuales están conectadas por resortes $i$ e $i+1$ con coeficientes de rigidez $K^i$ y $K^{i+1}$. De manera similar, las partículas tienen asignados los nombres $j-1$, $j$ y $j+1$ y sus respectivos desplazamientos se denotan por $u_{j-1}$, $u_{j}$ y $u_{j+1}$.

<center>
    <img src="img/ibc.svg"
         alt="files"
         style="width:300px">
</center>

Escribamos las ecuaciones de equilibrio para los resortes $i$ e $i+1$ en términos de los desplazamientos $u_{j - 1}$, $u_j$ y $u_{j + 1}$ como:

$$\begin{Bmatrix} f_1^i\\ f_2^i \end{Bmatrix}
= \begin{bmatrix}
  k_{11}^i & k_{12}^i\\
  k_{21}^i & k_{22}^i
\end{bmatrix}
\begin{Bmatrix} u_{j - 1}\\ u_j \end{Bmatrix}\, ,$$

y

$$\begin{Bmatrix} f_1^{i + 1}\\ f_2^{i + 1}\end{Bmatrix}
= \begin{bmatrix}
  k_{11}^{i + 1} & k_{12}^{i + 1}\\
  k_{21}^{i + 1} & k_{22}^{i + 1} \end{bmatrix}
\begin{Bmatrix} u_j\\ u_{j + 1} \end{Bmatrix}\, .$$

Note que hemos usado notación fila-columna en los índices para los coeficientes de rigidez para simplificar su implementación en el computador.

### Equilibrio de una partícula

Asumiendo que la partícula $j$ se encuentra sometida a una carga externa $P$, se tiene, de su diagrama de cuerpo libre, que:

$$
k_{21}^i u_{j - 1} + (k_{22}^i + k_{11}^{i + 1}) u_j + k_{12}^{i + 1} u_{j + 1} = P_j.
$$

Procediendo de manera similar para las partículas ${j-1}$ y ${j+1}$ y considerando la contribución de los resortes $K^i$ y $K^{i+1}$ obtenemos el siguiente bloque del sistema completo de ecuaciones:

$$\begin{bmatrix}
  k_{11}^i &k_{12}^i &0\\
  k_{21}^i &k_{22}^i + k_{11}^{i + 1} &k_{12}^{i + 1}\\
  0 &k_{21}^{i + 1} &k_{22}^{i + 1}
\end{bmatrix}\, . $$

Al considerar el sistema completo de partículas y resortes se obtiene un sistema de ecuaciones lineales de la forma general:

$$
[K_G] \{U_G\}  = \{F_G\}\, .
$$

En este sistema cada ecuación representa el equilibrio de una partícula.


### Ensamblaje

La construcción de las matrices globales que describen el equilibrio de cada masa (o partícula) en el sistema puede programarse en el computador mediante un algoritmo que considere el aporte (en términos de fuerzas) de cada elemento (o resorte).

En Mecánica Computacional, a este proceso se le denomina como **Ensamblaje** de las ecuaciones globales. La operación de ensamblaje puede realizarse después de identificar la conexión entre los (desplazamientos o) grados de libertad globales y los (desplazamientos o) grados de libertad locales mediante una matriz que almacena en cada fila los identificadores de los grados de libertad globales asociados a cada elemento.

Por ejemplo, en el sistema de 3-partículas los resortes $i$ y $i+1$ tienen como desplazamientos de sus extremos los denotados como $u_{j-1}$ y $u_j$ y $u_j$ y $u_{j+1}$. Dichos índices contienen toda la información necesaria para realizar el ensamblaje. La matriz que almacena los indices globales para todos los elementos en el modelo es la matriz `DME` dada por:

$$ DME = \begin{bmatrix}
  j - 1 &j\\
  j &j + 1\\
\end{bmatrix}\, .$$

Nótese que en esta matriz los datos de la primera fila mostrada son $j - 1$ y $j$, los cuales corresponden a los identificadores de los grados de libertad asociados con el resorte $i$. De la misma manera los datos de la segunda fila mostrada son $j$ y $j+1$ los cuales corresponden a los identificadores de los grados de libertad asociados con el resorte $i+1$.

Una vez se tiene disponible la matriz `DME`, el proceso de ensamblaje se realiza como se describe a continuación:

\begin{align}
&K_{j - 1,j - 1} \leftarrow K_{j - 1,j - 1} + k_{11}^i\\
&K_{j - 1,j} \leftarrow K_{j - 1,j} + k_{12}^i\\
&K_{j,j - 1} \leftarrow K_{j,j - 1} + k_{21}^i\\
&K_{j,j} \leftarrow K_{j,j} + k_{22}^i
\end{align}

y

\begin{align}
&K_{j,j} \leftarrow K_{j,j} + k_{11}^{i + 1}\\
&K_{j,j + 1} \leftarrow K_{j,j + 1} + k_{12}^{i + 1}\\
&K_{j + 1,j} \leftarrow K_{j + 1,j} + k_{21}^{i + 1}\\
&K_{j + 1,j + 1} \leftarrow K_{j + 1,j + 1} + k_{22}^{i + 1}
\end{align}


Note la conexión entre los índices locales, correspondientes a $1$ y $2$, y las posiciones en la matriz global, correspondientes a $j-1$, $j$ y $j+1$.

## Programa simple de análisis estructural

De las secciones anteriores se concluye que el problema de equilibrio se resuelve tras ensamblar la contribución de cada resorte (o triángulo) en una matriz de rigidez global. En esta sección discutimos su solución en el computador. Para definir el modelo usaremos archivos de texto en los cuales indicaremos al programa las partículas que lo conforman, los resortes del sistema y su conexión con las partículas, la localización de las cargas externas y las propiedades de los resortes.

Inicialmente se describen las rutinas o funciones que conforman el programa y en la parte final estas son llamadas desde el programa principal.

El programa principal ejecuta los siguientes pasos:

* Lee el modelo.

* Calcula la matriz `DME`.

* Ensambla el sistema global de ecuaciones.

* Resuelve el sistema para determinar los desplazamientos globales $UG$.

(Los archivos de texto con los datos de entrada para las partículas, elementos, coeficientes de rigidez y cargas de este problema están almacenados en la carpeta `files` de este repositorio).

In [1]:
import numpy as np

 La rutina `readin()` lee los archivos de entrada que se encuentran en la carpeta `files` del respositorio y que deben ser almacenados localmente bajo esta misma estructura.

In [2]:
def readin():
    nodes = np.loadtxt('files/sprnodes.txt', ndmin=2)
    mats = np.loadtxt('files/sprmater.txt', ndmin=2)
    elements = np.loadtxt('files/spreles.txt' , ndmin=2, dtype=np.int)
    loads = np.loadtxt('files/sprloads.txt', ndmin=2)
    return nodes, mats, elements, loads

En el archivo de nodos, en el cual se almacena la información de cada partícula, hay un código con valor $-1$ o $0$ indicando si la partícula se encuentra restringida (como en el empotramiento) o libre. Con esa información la rutina `eqcounter()` asigna identificadores de grados de libertad o números de ecuaciones a cada una de las partículas declaradas como libres.

<div class="alert alert-warning">
Agregue un comentario a cada línea relevante del código y use los comentarios para escribir el pseudocódigo de la función.    
</div>

In [3]:
def eqcounter(nodes):
    nn = nodes.shape[0]
    IBC = np.zeros((nn, 1), dtype=np.integer)
    neq = 0
    for cont in range(nn):
        IBC[cont] = int(nodes[cont, 2])
        if IBC[cont] == 0:
            IBC[cont] = neq
            neq = neq + 1
    return neq, IBC

El numero de ecuación asignado a cada partícula es usado ahora para formar la matriz `DME`. Cada fila de esta matriz contiene los identificadores de los desplazamientos de los extremos del resorte.

<div class="alert alert-warning">
Agregue un comentario a cada línea relevante del código y use los comentarios para escribir el pseudocódigo de la función.    
</div>

In [4]:
def DME(nodes, elements):
    nels = elements.shape[0]
    DME_mat = np.zeros((nels, 2), dtype=np.integer)
    neq, IBC = eqcounter(nodes)
    ndof = 2
    nnodes = 2
    for ele in range(nels):
        for node in range(nnodes):
            pos = elements[ele, node + 3]
            DME_mat[ele, node] = IBC[pos]
    return DME_mat, IBC, neq

Usando la matriz `DME` es posible ensamblar la matriz global de coeficientes de rigidez en términos de ecuaciones como:

$$
K_{j - 1,j - 1} \leftarrow K_{j - 1,j - 1} + k_{11}^i\, .\\
$$

In [5]:
def assembly(elements, mats, nodes, neq, DME_mat):
    IELCON = np.zeros([2], dtype=np.integer)
    KG = np.zeros((neq, neq))
    nels = elements.shape[0]
    nnodes = 2
    ndof = 2
    for el in range(nels):
        elcoor = np.zeros((nnodes))
        im = np.int(elements[el, 2])
        par = mats[im]
        for j in range(nnodes):
            IELCON[j] = elements[el, j+3]
            elcoor[j] = nodes[IELCON[j], 1]
        kloc = uelspring(par[0])
        dme = DME_mat[el, :ndof]
        for row in range(ndof):
            glob_row = dme[row]
            if glob_row != -1:
                for col in range(ndof):
                    glob_col = dme[col]
                    if glob_col != -1:
                        KG[glob_row, glob_col] = KG[glob_row, glob_col] +\
                                                 kloc[row, col]

    return KG

La función `uelspring` calcula la matriz de rigidez de un resorte típico, mientras que la función `loadassem` ensambla el vector de cargas externas aplicadas sobre las partículas.

<div class="alert alert-warning">
Agregue un comentario a cada línea relevante de los códigos de las siguientes función y úselos para escribir los pseudocódigos correspondientes.    
</div>

In [6]:
def uelspring(kcof):
    """
    Esta rutina calcula la matriz de rigidez para
    un elemento tipo resorte conformado por 2 nodos.

    Parámetros
    ----------
    kcof : float
      Coeficiente de rigidez del resorte (>0).

    Retorna
    -------
    kloc : ndarray
      Matriz de coeficientes de rigidez local para
      el elemento tipo resorte (2, 2).


    """
    kloc = np.array([
        [kcof, -kcof],
        [-kcof, kcof]])
    return kloc

In [7]:
def loadasem(loads, IBC, neq, nl):
    """
    Ensambla el vector de cargas o de valores conocidos
    en el sistema global de ecuaciones.

    Parámetros
    ----------
    loads : ndarray
      Arreglo con las cargas impuestas a las partículas.
    IBC : ndarray (int)
      Arreglo que almacena el identificador de grado de libertad
      a cada partícula.
    neq : int
      Numero de ecuaciones en el sistema.
    nl : int
      Numero de cargas en el sistema.

    Retorna
    -------
    rhs_glob : ndarray
      Arreglo con las cargas impuestas a las partículas,
      rhs se refiere a lado derecho (del inglés right-hand-side).

    """
    rhs_glob = np.zeros((neq))
    for cont in range(nl):
        il = int(loads[cont, 0])
        ilx = IBC[il]
        if ilx != -1:
            rhs_glob[ilx] = loads[cont, 1]
    return rhs_glob

### Programa principal

In [8]:
nodes, mats, elements, loads = readin()
DME, IBC, neq = DME(nodes, elements)
KG = assembly(elements, mats, nodes, neq, DME)
RHSG = loadasem(loads, IBC, neq, 3)
UG = np.linalg.solve(KG, RHSG)
print(UG)

[0.002  0.0025 0.0045]


<div class="alert alert-warning">

### Actividades para resolver con el computador

* Usando los desplazamientos encontrados en la solución del sistema determine las fuerzas resultantes en cada resorte e indique que elementos están a compresión y que elementos están a tensión.

* Usando las fuerzas encontradas en el paso anterior verifique el equilibrio del sistema.

* Libere el punto correspondiente al empotramiento y calcule nuevamente la solución. Discuta sus resultados.

* Resuelva nuevamente el sistema haciendo la rigidez del resorte mas a la derecha igual a $5k$. Discuta como varían las fuerzas internas en comparación con las del sistema original.

</div>

### Glosario de términos

**Coeficiente de rigidez:** Fuerza experimentada por un resorte empotrado en su extremo y sometido en su extremo opuesto a un desplazamiento unitario.

**Matriz de rigidez:** Matriz cuyos elementos son coeficientes de rigidez relacionando las fuerzas y los desplazamientos de un sistema de partículas.

**Sistema discreto:** Sistema mecánico cuyas ecuaciones de equilibrio son representables de manera directa mediante relaciones fuerza-desplazamiento por medio de una matriz de rigidez.

**Sistema continuo:** Sistema mecánico cuyas ecuaciones de equilibrio corresponden a ecuaciones diferenciales parciales que se satisfacen sobre el dominio del sistema.

**Ensamblaje:** Operación mediante la cual se considera el aporte de un sistema de resortes a la rigidez de todo el sistema.

In [9]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = 'estilo.css'
HTML(open(css_file, "r").read())